<a href="https://colab.research.google.com/github/KavinRV/DSML_Project/blob/main/DSML_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
import random
import pandas as pd
import seaborn as sns
import numpy as np 
from pprint import pprint
import matplotlib.pyplot as plt

from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline as imb_pipe

import plotly.express as px
import plotly.graph_objects as go 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_validate

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

from sklearn.pipeline import Pipeline

In [2]:
train_label = pd.read_csv("/content/drive/MyDrive/ml_class_materials/project_data/project16/glitch_trn_class_labels.csv", header=None)
train_data = pd.read_csv("/content/drive/MyDrive/ml_class_materials/project_data/project16/glitch_trn_data.csv")
test_data = pd.read_csv("/content/drive/MyDrive/ml_class_materials/project_data/project16/glitch_tst_data.csv")

In [3]:
train_label.head()

,0,1
0,d1,Blip
1,d2,Extremely_Loud
2,d3,Violin_Mode
3,d4,Blip
4,d5,Scratchy


In [4]:
train_data.head()

,id,GPStime,centralFreq,peakFreq,snr,ifo,bandwidth,duration
0,HH9ABflByx,1.129517e+09,364.822,225.349,35.919,L1,665.643311,0.250
1,oTXUV2ynKq,1.135647e+09,3609.954,38.027,2743.555,H1,7209.994629,13.690
2,jQwe6arh9F,1.129802e+09,1838.466,2025.310,16.698,H1,3612.932861,0.375
3,g8bOTBroX2,1.131774e+09,2067.878,111.128,12.727,H1,4067.436523,0.105
4,7KAvyGygKM,1.130969e+09,3978.111,131.921,10.926,L1,7795.106934,2.304


In [5]:
test_data.head()

,id,GPStime,centralFreq,peakFreq,snr,ifo,bandwidth,duration
0,YVnrMXRcUP,1.132261e+09,2960.810,38.027,63.400,H1,5902.753418,2.000
1,qD8uxo1Jeu,1.132725e+09,1119.993,324.754,11.789,H1,2135.070557,0.070
2,Z5grpD4TOG,1.128422e+09,236.863,134.965,8.000,L1,211.306503,0.047
3,4TskgvMLa2,1.132886e+09,1905.332,137.711,14.362,H1,3742.343506,0.125
4,ZMEZmAs4gO,1.132755e+09,1119.433,170.654,9.087,H1,2174.645996,0.230


In [6]:
train_data["ifo"].unique()

array(['L1', 'H1'], dtype=object)

#Preprocessing

In [7]:
le = LabelEncoder()
le.fit(train_data.ifo)
train_data['ifo_num'] = le.transform(train_data.ifo)
train_data.head()

,id,GPStime,centralFreq,peakFreq,snr,ifo,bandwidth,duration,ifo_num
0,HH9ABflByx,1.129517e+09,364.822,225.349,35.919,L1,665.643311,0.250,1
1,oTXUV2ynKq,1.135647e+09,3609.954,38.027,2743.555,H1,7209.994629,13.690,0
2,jQwe6arh9F,1.129802e+09,1838.466,2025.310,16.698,H1,3612.932861,0.375,0
3,g8bOTBroX2,1.131774e+09,2067.878,111.128,12.727,H1,4067.436523,0.105,0
4,7KAvyGygKM,1.130969e+09,3978.111,131.921,10.926,L1,7795.106934,2.304,1


In [69]:
le = LabelEncoder()
le.fit(test_data.ifo)
test_data['ifo_num'] = le.transform(test_data.ifo)
test_data.head()

,id,GPStime,centralFreq,peakFreq,snr,ifo,bandwidth,duration,ifo_num
0,YVnrMXRcUP,1.132261e+09,2960.810,38.027,63.400,H1,5902.753418,2.000,0
1,qD8uxo1Jeu,1.132725e+09,1119.993,324.754,11.789,H1,2135.070557,0.070,0
2,Z5grpD4TOG,1.128422e+09,236.863,134.965,8.000,L1,211.306503,0.047,1
3,4TskgvMLa2,1.132886e+09,1905.332,137.711,14.362,H1,3742.343506,0.125,0
4,ZMEZmAs4gO,1.132755e+09,1119.433,170.654,9.087,H1,2174.645996,0.230,0


In [73]:
le = LabelEncoder()
le.fit(train_label[1])
train_label['label'] = le.transform(train_label[1])
train_label.head()

,0,1,label
0,d1,Blip,3
1,d2,Extremely_Loud,5
2,d3,Violin_Mode,19
3,d4,Blip,3
4,d5,Scratchy,17


In [9]:
train_data['label'] = train_label['label']


In [10]:
for col in train_data.columns[:-1]:
    print(train_data[[col, "label"]].groupby('label').describe())

         id                        
      count unique         top freq
label                              
0         4      4  72FpqgxwTs    1
1        75     75  7derEpwLIL    1
2        51     51  hB93a6UPPp    1
3      1587   1587  HH9ABflByx    1
4        54     54  3FcGoZua0h    1
5       403    403  oTXUV2ynKq    1
6       243    243  5lL68EymOk    1
7       638    638  KJXiJjU9R5    1
8       360    360  3eWwicd16b    1
9       474    474  iH9g1mcRF7    1
10      445    445  IwZrYzVNpy    1
11       37     37  cwPkRt3R8p    1
12      136    136  YU3iEjH75G    1
13       23     23  N1kAm9Ytrb    1
14      405    405  2OS69pGExE    1
15       82     82  cIb4LRGlLH    1
16      384    384  qU8GHCsHWN    1
17      242    242  7KAvyGygKM    1
18       84     84  gdV6DYhWrb    1
19      123    123  jQwe6arh9F    1
20       19     19  wgQz9Eq3OP    1
21      131    131  Uo1TnuQ2KM    1
      GPStime                                                          \
        count          mean

#Initial Training on Imbalanced Data


##Decision Tree

In [11]:
dt_mod_unb = DecisionTreeClassifier()
X = train_data[["GPStime", "centralFreq", "peakFreq", "snr", "bandwidth", "duration", "ifo_num"]]
scores = cross_validate(dt_mod_unb, X, train_label['label'], n_jobs=-1, scoring="f1_weighted")

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,


In [12]:
scores

{'fit_time': array([0.05097032, 0.05115199, 0.0527513 , 0.06884909, 0.05824304]),
 'score_time': array([0.00596499, 0.00784779, 0.00607276, 0.00511742, 0.00556135]),
 'test_score': array([0.83287857, 0.83087895, 0.80734079, 0.81072688, 0.82317761])}

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, train_label['label'], test_size=0.2, random_state=0)

dtc_model = DecisionTreeClassifier()
clf = dtc_model.fit(X_train, y_train)

pred_labels = dtc_model.predict(X_test)

print('Classes: ', clf.classes_) 
# print('Class Priors: ',clf.class_prior_)

print('-----------------------Decision Tree----------------------')
score = dtc_model.score(X_test, y_test)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
-----------------------Decision Tree----------------------
Accuracy Score:  0.8125
Weighted-F1: 0.8128271482427345
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.91      1.00      0.95        10
           2       0.91      0.83      0.87        12
           3       0.90      0.86      0.88       314
           4       0.40      0.50      0.44         8
           5       0.78      0.87      0.82        83
           6       0.86      0.96      0.91        52
           7       0.83      0.82      0.83       122
           8       0.58      0.64      0.61        70
           9       0.78      0.75      0.76       103
          10       0.78      0.78      0.78        85
          11       0.29      0.33      0.31         6
          12       0.48      0.38      0.43    

In [14]:
criterion = ['gini', 'entropy']
max_depth = [x for x in range(2, 30, 2)]

std_slc = StandardScaler()

pipe_dtc = Pipeline(steps=[('std_slc', std_slc),
                       ('dec_tree', DecisionTreeClassifier())])

param_grid_gtc = dict(dec_tree__criterion=criterion, dec_tree__max_depth=max_depth)

In [15]:
gtc_Model_grid = GridSearchCV(estimator=pipe_dtc, param_grid=param_grid_gtc, verbose=1, cv=10, n_jobs=-1, scoring='f1_weighted')
gtc_Model_grid.fit(X_train, y_train)
print(gtc_Model_grid.best_estimator_)
print(f'Best Score: {gtc_Model_grid.best_score_}')

Fitting 10 folds for each of 28 candidates, totalling 280 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  UserWarning,


Pipeline(steps=[('std_slc', StandardScaler()),
                ('dec_tree',
                 DecisionTreeClassifier(criterion='entropy', max_depth=14))])
Best Score: 0.8350832898285658


In [16]:
print('Best Criterion:', gtc_Model_grid.best_estimator_.get_params()['dec_tree__criterion'])
print('Best max_depth:', gtc_Model_grid.best_estimator_.get_params()['dec_tree__max_depth'])
print(); print(gtc_Model_grid.best_estimator_.get_params()['dec_tree'])

Best Criterion: entropy
Best max_depth: 14

DecisionTreeClassifier(criterion='entropy', max_depth=14)


In [17]:
pipe_gtc = gtc_Model_grid.best_estimator_
dtc_model = pipe_gtc
clf = dtc_model.fit(X_train, y_train)

pred_labels = dtc_model.predict(X_test)

print('Classes: ', clf.classes_) 
# print('Class Priors: ',clf.class_prior_)

print('-----------------------Decision Tree----------------------')
score = dtc_model.score(X_test, y_test)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
-----------------------Decision Tree----------------------
Accuracy Score:  0.8275
Weighted-F1: 0.8280283899577551
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00        10
           2       1.00      0.83      0.91        12
           3       0.91      0.86      0.88       314
           4       0.33      0.38      0.35         8
           5       0.83      0.82      0.82        83
           6       0.82      0.96      0.88        52
           7       0.82      0.87      0.84       122
           8       0.59      0.69      0.64        70
           9       0.77      0.83      0.79       103
          10       0.85      0.74      0.79        85
          11       0.25      0.17      0.20         6
          12       0.56      0.58      0.57    

In [18]:
print(clf['dec_tree'].feature_importances_)
clf.feature_names_in_

[0.09097651 0.02408289 0.46261975 0.24628856 0.02308586 0.10439785
 0.04854859]


array(['GPStime', 'centralFreq', 'peakFreq', 'snr', 'bandwidth',
       'duration', 'ifo_num'], dtype=object)

So from above we can see "GPStime", "peakFreq", "snr", "duration" are more importan feature than the others

In [19]:
X_red = train_data[["GPStime", "peakFreq", "snr", "duration", "ifo_num"]]

X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(X_red, train_label['label'], test_size=0.2, random_state=0)

dtc_model = DecisionTreeClassifier()
clf = dtc_model.fit(X_train_red, y_train_red)

pred_labels = dtc_model.predict(X_test_red)

print('Classes: ', clf.classes_) 
# print('Class Priors: ',clf.class_prior_)

print('--------------------Decision Tree Red-------------------')
score = dtc_model.score(X_test_red, y_test_red)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_red, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test_red, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
--------------------Decision Tree Red-------------------
Accuracy Score:  0.8325
Weighted-F1: 0.8314173041600624
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.91      1.00      0.95        10
           2       0.80      0.67      0.73        12
           3       0.92      0.89      0.90       314
           4       0.70      0.88      0.78         8
           5       0.83      0.86      0.84        83
           6       0.93      1.00      0.96        52
           7       0.86      0.87      0.87       122
           8       0.59      0.69      0.63        70
           9       0.78      0.76      0.77       103
          10       0.78      0.80      0.79        85
          11       0.25      0.17      0.20         6
          12       0.53      0.38      0.44      

In [20]:
criterion = ['gini', 'entropy']
max_depth = [x for x in range(2, 30, 2)]

std_slc = StandardScaler()

pipe_dtc = Pipeline(steps=[('std_slc', std_slc),
                       ('dec_tree', DecisionTreeClassifier())])

param_grid_gtc = dict(dec_tree__criterion=criterion, dec_tree__max_depth=max_depth)

In [21]:
gtc_Model_grid = GridSearchCV(estimator=pipe_dtc, param_grid=param_grid_gtc, verbose=1, cv=10, n_jobs=-1, scoring='f1_weighted')
gtc_Model_grid.fit(X_train_red, y_train_red)
print(gtc_Model_grid.best_estimator_)
print(f'Best Score: {gtc_Model_grid.best_score_}')

Fitting 10 folds for each of 28 candidates, totalling 280 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  UserWarning,


Pipeline(steps=[('std_slc', StandardScaler()),
                ('dec_tree',
                 DecisionTreeClassifier(criterion='entropy', max_depth=12))])
Best Score: 0.830314994032614


In [22]:
print('Best Criterion:', gtc_Model_grid.best_estimator_.get_params()['dec_tree__criterion'])
print('Best max_depth:', gtc_Model_grid.best_estimator_.get_params()['dec_tree__max_depth'])
print(); print(gtc_Model_grid.best_estimator_.get_params()['dec_tree'])

Best Criterion: entropy
Best max_depth: 12

DecisionTreeClassifier(criterion='entropy', max_depth=12)


In [23]:
pipe = gtc_Model_grid.best_estimator_
dtc_model = pipe
clf = dtc_model.fit(X_train_red, y_train_red)

pred_labels = dtc_model.predict(X_test_red)

print('Classes: ', clf.classes_) 
# print('Class Priors: ',clf.class_prior_)

print('---------------Decision Tree Red Best-------------')
score = dtc_model.score(X_test_red, y_test_red)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_red, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test_red, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
---------------Decision Tree Red Best-------------
Accuracy Score:  0.8458333333333333
Weighted-F1: 0.842216234536807
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00        10
           2       0.91      0.83      0.87        12
           3       0.89      0.93      0.91       314
           4       0.43      0.38      0.40         8
           5       0.81      0.84      0.83        83
           6       0.94      0.96      0.95        52
           7       0.88      0.89      0.89       122
           8       0.58      0.54      0.56        70
           9       0.82      0.81      0.81       103
          10       0.82      0.78      0.80        85
          11       0.00      0.00      0.00         6
          12       0.67      0.62      0.64 

Even after removing some features the modle is doing similar or better than to one with all features.
So will be using this reduced feature for the future clasification

##K Nearest Neighbour

In [24]:
knn_pipe = Pipeline(steps=[('std_slc', std_slc),
                       ('knn', KNeighborsClassifier())])

In [25]:
clf = knn_pipe.fit(X_train_red, y_train_red)

pred_labels = knn_pipe.predict(X_test_red)

print('Classes: ', clf.classes_) 

print('------------------------KNN Red-------------------------')
score = knn_pipe.score(X_test_red, y_test_red)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_red, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test_red, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
------------------------KNN Red-------------------------
Accuracy Score:  0.7258333333333333
Weighted-F1: 0.7118830351437504
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00        10
           2       0.77      0.83      0.80        12
           3       0.83      0.89      0.86       314
           4       0.62      0.62      0.62         8
           5       0.79      0.80      0.79        83
           6       0.96      0.98      0.97        52
           7       0.68      0.71      0.70       122
           8       0.51      0.61      0.55        70
           9       0.55      0.57      0.56       103
          10       0.57      0.47      0.52        85
          11       0.25      0.17      0.20         6
          12       0.17      0.08    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
knn_pipe.get_params()

{'knn': KNeighborsClassifier(),
 'knn__algorithm': 'auto',
 'knn__leaf_size': 30,
 'knn__metric': 'minkowski',
 'knn__metric_params': None,
 'knn__n_jobs': None,
 'knn__n_neighbors': 5,
 'knn__p': 2,
 'knn__weights': 'uniform',
 'memory': None,
 'std_slc': StandardScaler(),
 'std_slc__copy': True,
 'std_slc__with_mean': True,
 'std_slc__with_std': True,
 'steps': [('std_slc', StandardScaler()), ('knn', KNeighborsClassifier())],
 'verbose': False}

In [27]:
param_grid_knn = [{
    'knn__leaf_size': list(range(1, 15)),
    'knn__n_neighbors': list(range(1, 9)),
    'knn__p': [1, 2],
}]

In [28]:
knnModel_grid = GridSearchCV(estimator=knn_pipe, param_grid=param_grid_knn, verbose=1, cv=10, n_jobs=-1, scoring='f1_weighted')
knnModel_grid.fit(X_train_red, y_train_red)
print(knnModel_grid.best_estimator_)
print(f'Best Score: {knnModel_grid.best_score_}')

Fitting 10 folds for each of 224 candidates, totalling 2240 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  UserWarning,


Pipeline(steps=[('std_slc', StandardScaler()),
                ('knn', KNeighborsClassifier(leaf_size=1, p=1))])
Best Score: 0.7391830447284395


In [29]:
knn_pipe = knnModel_grid.best_estimator_
clf = knn_pipe.fit(X_train_red, y_train_red)

pred_labels = knn_pipe.predict(X_test_red)

print('Classes: ', clf.classes_) 

print('---------------------KNN Red Best-----------------------')
score = knn_pipe.score(X_test_red, y_test_red)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_red, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test_red, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
---------------------KNN Red Best-----------------------
Accuracy Score:  0.7533333333333333
Weighted-F1: 0.741115320477189
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00        10
           2       0.91      0.83      0.87        12
           3       0.84      0.89      0.87       314
           4       0.71      0.62      0.67         8
           5       0.80      0.81      0.80        83
           6       0.96      0.98      0.97        52
           7       0.75      0.75      0.75       122
           8       0.57      0.71      0.64        70
           9       0.56      0.61      0.59       103
          10       0.58      0.53      0.56        85
          11       0.40      0.33      0.36         6
          12       0.38      0.19     

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Logistic Regression

In [30]:
lr_pipe = Pipeline(steps=[('std_slc', std_slc),
                       ('log_reg', LogisticRegression())])

In [31]:
clf = lr_pipe.fit(X_train_red, y_train_red)

pred_labels = lr_pipe.predict(X_test_red)

print('Classes: ', clf.classes_) 

print('----------------Logistic Regression Red-----------------')
score = lr_pipe.score(X_test_red, y_test_red)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_red, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test_red, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
----------------Logistic Regression Red-----------------
Accuracy Score:  0.6508333333333334
Weighted-F1: 0.6057615510973148
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.71      1.00      0.83        10
           2       0.00      0.00      0.00        12
           3       0.74      0.92      0.82       314
           4       0.00      0.00      0.00         8
           5       0.81      0.82      0.81        83
           6       0.58      1.00      0.73        52
           7       0.71      0.73      0.72       122
           8       0.33      0.10      0.15        70
           9       0.46      0.32      0.38       103
          10       0.49      0.52      0.51        85
          11       0.00      0.00      0.00         6
          12       0.00      0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0

In [32]:
lr_pipe.get_params()

{'log_reg': LogisticRegression(),
 'log_reg__C': 1.0,
 'log_reg__class_weight': None,
 'log_reg__dual': False,
 'log_reg__fit_intercept': True,
 'log_reg__intercept_scaling': 1,
 'log_reg__l1_ratio': None,
 'log_reg__max_iter': 100,
 'log_reg__multi_class': 'auto',
 'log_reg__n_jobs': None,
 'log_reg__penalty': 'l2',
 'log_reg__random_state': None,
 'log_reg__solver': 'lbfgs',
 'log_reg__tol': 0.0001,
 'log_reg__verbose': 0,
 'log_reg__warm_start': False,
 'memory': None,
 'std_slc': StandardScaler(),
 'std_slc__copy': True,
 'std_slc__with_mean': True,
 'std_slc__with_std': True,
 'steps': [('std_slc', StandardScaler()), ('log_reg', LogisticRegression())],
 'verbose': False}

In [33]:
param_lr_grid = {
    'log_reg__penalty' : ['l1','l2'], 
    'log_reg__C'       : np.logspace(0,6,7),
    'log_reg__solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
}

In [34]:
lrModel_grid = GridSearchCV(estimator=lr_pipe, param_grid=param_lr_grid, verbose=1, cv=10, n_jobs=-1, scoring='f1_weighted')
lrModel_grid.fit(X_train_red, y_train_red)
print(lrModel_grid.best_estimator_)
print(f'Best Score: {lrModel_grid.best_score_}')

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  UserWarning,


Fitting 10 folds for each of 42 candidates, totalling 420 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
140 fits failed out of a total of 420.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
70 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  F

Pipeline(steps=[('std_slc', StandardScaler()),
                ('log_reg',
                 LogisticRegression(C=100000.0, solver='newton-cg'))])
Best Score: 0.6831334792471521


/usr/local/lib/python3.7/dist-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  ConvergenceWarning,


In [35]:
lr_pipe = lrModel_grid.best_estimator_ 
clf = lr_pipe.fit(X_train_red, y_train_red)

pred_labels = lr_pipe.predict(X_test_red)

print('Classes: ', clf.classes_) 

print('--------------Logistic Regression Red Best--------------')
score = lr_pipe.score(X_test_red, y_test_red)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_red, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test_red, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
--------------Logistic Regression Red Best--------------
Accuracy Score:  0.72
Weighted-F1: 0.6918004367784854
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.77      1.00      0.87        10
           2       0.91      0.83      0.87        12
           3       0.82      0.92      0.87       314
           4       0.00      0.00      0.00         8
           5       0.86      0.80      0.82        83
           6       0.75      1.00      0.86        52
           7       0.82      0.80      0.81       122
           8       0.48      0.34      0.40        70
           9       0.49      0.32      0.39       103
          10       0.52      0.62      0.57        85
          11       0.00      0.00      0.00         6
          12       0.00      0.00      0.00        

/usr/local/lib/python3.7/dist-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. U

##Support Vector Machine

In [36]:
svm_pipe = Pipeline(steps=[('std_slc', std_slc),
                       ('svm_reg', SVC())])

In [37]:
clf = svm_pipe.fit(X_train_red, y_train_red)

pred_labels = svm_pipe.predict(X_test_red)

print('Classes: ', clf.classes_) 

print('-------------------------SVM Red------------------------')
score = svm_pipe.score(X_test_red, y_test_red)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_red, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test_red, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
-------------------------SVM Red------------------------
Accuracy Score:  0.6483333333333333
Weighted-F1: 0.6078028950971341
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.71      1.00      0.83        10
           2       0.00      0.00      0.00        12
           3       0.78      0.92      0.85       314
           4       0.00      0.00      0.00         8
           5       0.79      0.81      0.80        83
           6       0.60      1.00      0.75        52
           7       0.60      0.62      0.61       122
           8       0.38      0.39      0.38        70
           9       0.51      0.25      0.34       103
          10       0.44      0.59      0.51        85
          11       0.00      0.00      0.00         6
          12       0.00      0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
svm_pipe.get_params()

{'memory': None,
 'std_slc': StandardScaler(),
 'std_slc__copy': True,
 'std_slc__with_mean': True,
 'std_slc__with_std': True,
 'steps': [('std_slc', StandardScaler()), ('svm_reg', SVC())],
 'svm_reg': SVC(),
 'svm_reg__C': 1.0,
 'svm_reg__break_ties': False,
 'svm_reg__cache_size': 200,
 'svm_reg__class_weight': None,
 'svm_reg__coef0': 0.0,
 'svm_reg__decision_function_shape': 'ovr',
 'svm_reg__degree': 3,
 'svm_reg__gamma': 'scale',
 'svm_reg__kernel': 'rbf',
 'svm_reg__max_iter': -1,
 'svm_reg__probability': False,
 'svm_reg__random_state': None,
 'svm_reg__shrinking': True,
 'svm_reg__tol': 0.001,
 'svm_reg__verbose': False,
 'verbose': False}

In [45]:
svm_param_grid = {'svm_reg__C': [980, 985, 975],
              'svm_reg__gamma': [0.65, 0.55, 0.6],
              'svm_reg__kernel': ['rbf']}

In [46]:
svmModel_grid = GridSearchCV(estimator=svm_pipe, param_grid=svm_param_grid, verbose=1, cv=3, n_jobs=-1, scoring='f1_weighted')
svmModel_grid.fit(X_train_red, y_train_red)
print(svmModel_grid.best_estimator_)
print(f'Best Score: {svmModel_grid.best_score_}')

Fitting 3 folds for each of 9 candidates, totalling 27 fits
Pipeline(steps=[('std_slc', StandardScaler()),
                ('svm_reg', SVC(C=980, gamma=0.6))])
Best Score: 0.7885412755205561


In [47]:
svm_pipe = svmModel_grid.best_estimator_

clf = svm_pipe.fit(X_train_red, y_train_red)

pred_labels = svm_pipe.predict(X_test_red)

print('Classes: ', clf.classes_) 

print('-----------------------SVM Red Best---------------------')
score = svm_pipe.score(X_test_red, y_test_red)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_red, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test_red, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
-----------------------SVM Red Best---------------------
Accuracy Score:  0.8091666666666667
Weighted-F1: 0.7978478632170206
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.83      1.00      0.91        10
           2       0.91      0.83      0.87        12
           3       0.83      0.96      0.89       314
           4       0.83      0.62      0.71         8
           5       0.81      0.82      0.81        83
           6       0.96      1.00      0.98        52
           7       0.87      0.87      0.87       122
           8       0.60      0.54      0.57        70
           9       0.70      0.67      0.68       103
          10       0.66      0.62      0.64        85
          11       0.33      0.33      0.33         6
          12       0.59      0.50    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Random Forest

In [ ]:
rf_pipe = Pipeline(steps=[('std_slc', std_slc),
                       ('ran_forest', RandomForestClassifier())])

In [ ]:
print('Parameters currently in use:\n')
pprint(rf_pipe.get_params())

Parameters currently in use:

{'memory': None,
 'ran_forest': RandomForestClassifier(),
 'ran_forest__bootstrap': True,
 'ran_forest__ccp_alpha': 0.0,
 'ran_forest__class_weight': None,
 'ran_forest__criterion': 'gini',
 'ran_forest__max_depth': None,
 'ran_forest__max_features': 'auto',
 'ran_forest__max_leaf_nodes': None,
 'ran_forest__max_samples': None,
 'ran_forest__min_impurity_decrease': 0.0,
 'ran_forest__min_samples_leaf': 1,
 'ran_forest__min_samples_split': 2,
 'ran_forest__min_weight_fraction_leaf': 0.0,
 'ran_forest__n_estimators': 100,
 'ran_forest__n_jobs': None,
 'ran_forest__oob_score': False,
 'ran_forest__random_state': None,
 'ran_forest__verbose': 0,
 'ran_forest__warm_start': False,
 'std_slc': StandardScaler(),
 'std_slc__copy': True,
 'std_slc__with_mean': True,
 'std_slc__with_std': True,
 'steps': [('std_slc', StandardScaler()),
           ('ran_forest', RandomForestClassifier())],
 'verbose': False}


In [ ]:
clf = rf_pipe.fit(X_train_red, y_train_red)

pred_labels = rf_pipe.predict(X_test_red)

print('Classes: ', clf.classes_) 
# print('Class Priors: ',clf.class_prior_)

print('--------------------Random Forest Red-------------------')
score = rf_pipe.score(X_test_red, y_test_red)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_red, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test_red, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
--------------------Random Forest Red-------------------
Accuracy Score:  0.8258333333333333
Weighted-F1: 0.880426883024533
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00        10
           2       0.83      0.83      0.83        12
           3       0.90      0.96      0.93       314
           4       0.78      0.88      0.82         8
           5       0.85      0.86      0.85        83
           6       0.96      1.00      0.98        52
           7       0.89      0.93      0.91       122
           8       0.74      0.71      0.72        70
           9       0.86      0.84      0.85       103
          10       0.88      0.84      0.86        85
          11       0.50      0.17      0.25         6
          12       0.76      0.50     

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


We Could Already see quite a bit of improvement after using random forest.

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 6, 8, 10]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'ran_forest__n_estimators': n_estimators,
               'ran_forest__max_features': max_features,
               'ran_forest__max_depth': max_depth,
               'ran_forest__min_samples_split': min_samples_split,
               'ran_forest__min_samples_leaf': min_samples_leaf,
               'ran_forest__bootstrap': bootstrap}
pprint(random_grid)

{'ran_forest__bootstrap': [True, False],
 'ran_forest__max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'ran_forest__max_features': ['auto', 'sqrt'],
 'ran_forest__min_samples_leaf': [1, 2, 4, 6, 8, 10],
 'ran_forest__min_samples_split': [2, 5, 10],
 'ran_forest__n_estimators': [100,
                              311,
                              522,
                              733,
                              944,
                              1155,
                              1366,
                              1577,
                              1788,
                              2000]}


In [ ]:
rf_random = RandomizedSearchCV(estimator = rf_pipe, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=1, n_jobs = -1, scoring="f1_weighted")
rf_random.fit(X_train_red, y_train_red)
print(rf_random.best_estimator_)
print(f'Best Score: {rf_random.best_score_}')

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Pipeline(steps=[('std_slc', StandardScaler()),
                ('ran_forest',
                 RandomForestClassifier(max_depth=96, max_features='sqrt',
                                        min_samples_split=5,
                                        n_estimators=1450))])
Best Score: 0.8728036550515225


In [ ]:
rf_pipe = Pipeline(steps=[('std_slc', StandardScaler()),
                ('ran_forest',
                 RandomForestClassifier(max_depth=96, max_features='sqrt',
                                        min_samples_split=5,
                                        n_estimators=1450))])
clf = rf_pipe.fit(X_train_red, y_train_red)

pred_labels = rf_pipe.predict(X_test_red)

print('Classes: ', clf.classes_) 
# print('Class Priors: ',clf.class_prior_)

print('--------------------Random Forest Red Best-------------------')
score = rf_pipe.score(X_test_red, y_test_red)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_red, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test_red, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
--------------------Random Forest Red Best-------------------
Accuracy Score:  0.8941666666666667
Weighted-F1: 0.8870805051716689
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00        10
           2       0.83      0.83      0.83        12
           3       0.90      0.96      0.93       314
           4       0.78      0.88      0.82         8
           5       0.84      0.86      0.85        83
           6       0.96      1.00      0.98        52
           7       0.90      0.93      0.92       122
           8       0.76      0.71      0.74        70
           9       0.86      0.84      0.85       103
          10       0.88      0.85      0.86        85
          11       1.00      0.17      0.29         6
          12       0.88      0.5

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [85]:
rf_pipe = Pipeline(steps=[('std_slc', StandardScaler()),
                ('ran_forest',
                 RandomForestClassifier(max_depth=96, max_features='sqrt',
                                        min_samples_split=5,
                                        n_estimators=1450, class_weight="balanced"))])
clf = rf_pipe.fit(X_train_red, y_train_red)

pred_labels = rf_pipe.predict(X_test_red)

print('Classes: ', clf.classes_) 
# print('Class Priors: ',clf.class_prior_)

print('--------------------Random Forest Red Best-------------------')
score = rf_pipe.score(X_test_red, y_test_red)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_red, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test_red, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
--------------------Random Forest Red Best-------------------
Accuracy Score:  0.895
Weighted-F1: 0.8897593115634486
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00        10
           2       0.83      0.83      0.83        12
           3       0.92      0.95      0.93       314
           4       0.89      1.00      0.94         8
           5       0.83      0.86      0.84        83
           6       0.98      1.00      0.99        52
           7       0.89      0.92      0.90       122
           8       0.72      0.80      0.76        70
           9       0.89      0.83      0.86       103
          10       0.88      0.86      0.87        85
          11       1.00      0.17      0.29         6
          12       0.74      0.54      0.62  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Training on a Oversampled Data

In [24]:
oversample = SMOTE(k_neighbors=3)
X_ove, y_ove = oversample.fit_resample(pd.concat([X_train_red, X_test_red]), pd.concat([y_train_red, y_test_red]))

In [25]:
counter = Counter(y_ove)
for i, j in counter.items():
    pct = j/len(y_ove) * 100
    print(f"Class = {i}, n = {j} ({pct})%")


Class = 16, n = 1587 (4.545454545454546)%
Class = 12, n = 1587 (4.545454545454546)%
Class = 3, n = 1587 (4.545454545454546)%
Class = 8, n = 1587 (4.545454545454546)%
Class = 7, n = 1587 (4.545454545454546)%
Class = 14, n = 1587 (4.545454545454546)%
Class = 19, n = 1587 (4.545454545454546)%
Class = 9, n = 1587 (4.545454545454546)%
Class = 5, n = 1587 (4.545454545454546)%
Class = 17, n = 1587 (4.545454545454546)%
Class = 10, n = 1587 (4.545454545454546)%
Class = 6, n = 1587 (4.545454545454546)%
Class = 15, n = 1587 (4.545454545454546)%
Class = 18, n = 1587 (4.545454545454546)%
Class = 11, n = 1587 (4.545454545454546)%
Class = 2, n = 1587 (4.545454545454546)%
Class = 21, n = 1587 (4.545454545454546)%
Class = 1, n = 1587 (4.545454545454546)%
Class = 4, n = 1587 (4.545454545454546)%
Class = 20, n = 1587 (4.545454545454546)%
Class = 13, n = 1587 (4.545454545454546)%
Class = 0, n = 1587 (4.545454545454546)%


In [26]:
X_train_ove, X_test_ove, y_train_ove, y_test_ove = train_test_split(X_ove, y_ove, test_size=0.2, random_state=0)

##Decision Tree

In [60]:
dtc_model = DecisionTreeClassifier()
clf = dtc_model.fit(X_train_ove, y_train_ove)

pred_labels = dtc_model.predict(X_test_ove)

print('Classes: ', clf.classes_) 
# print('Class Priors: ',clf.class_prior_)

print('-----------------------Decision Tree----------------------')
score = dtc_model.score(X_test_ove, y_test_ove)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_ove, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test_ove, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
-----------------------Decision Tree----------------------
Accuracy Score:  0.9497350708864385
Weighted-F1: 0.9495933668152794
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       337
           1       1.00      1.00      1.00       325
           2       0.99      0.99      0.99       300
           3       0.91      0.85      0.88       315
           4       0.99      0.99      0.99       319
           5       0.94      0.93      0.93       309
           6       0.98      0.99      0.99       322
           7       0.92      0.93      0.92       327
           8       0.88      0.85      0.87       306
           9       0.85      0.86      0.86       321
          10       0.87      0.87      0.87       323
          11       0.93      0.96      0.94       280
          12       0.87      0.89  

In [61]:
gtc_Model_grid = GridSearchCV(estimator=pipe_dtc, param_grid=param_grid_gtc, verbose=1, cv=10, n_jobs=-1, scoring='f1_weighted')
gtc_Model_grid.fit(X_train_ove, y_train_ove)
print(gtc_Model_grid.best_estimator_)
print(f'Best Score: {gtc_Model_grid.best_score_}')

Fitting 10 folds for each of 28 candidates, totalling 280 fits
Pipeline(steps=[('std_slc', StandardScaler()),
                ('dec_tree',
                 DecisionTreeClassifier(criterion='entropy', max_depth=28))])
Best Score: 0.9432564243140666


In [63]:
pipe_gtc = gtc_Model_grid.best_estimator_
clf = pipe_gtc.fit(X_train_ove, y_train_ove)

pred_labels = pipe_gtc.predict(X_test_ove)

print('Classes: ', clf.classes_) 
# print('Class Priors: ',clf.class_prior_)

print('-----------------------Decision Tree----------------------')
score = dtc_model.score(X_test_ove, y_test_ove)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_ove, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test_ove, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
-----------------------Decision Tree----------------------
Accuracy Score:  0.9497350708864385
Weighted-F1: 0.9478578629520836
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       337
           1       1.00      1.00      1.00       325
           2       0.98      1.00      0.99       300
           3       0.87      0.83      0.85       315
           4       0.98      0.99      0.99       319
           5       0.92      0.92      0.92       309
           6       1.00      0.99      0.99       322
           7       0.89      0.88      0.89       327
           8       0.89      0.88      0.88       306
           9       0.88      0.90      0.89       321
          10       0.91      0.89      0.90       323
          11       0.94      0.95      0.94       280
          12       0.88      0.86  

##K Nearest Neighbour

In [30]:
knn_pipe = Pipeline(steps=[('std_slc', std_slc),
                       ('knn', KNeighborsClassifier())])

In [31]:
clf = knn_pipe.fit(X_train_ove, y_train_ove)

pred_labels = knn_pipe.predict(X_test_ove)

print('Classes: ', clf.classes_)

print('-----------------------KNN Over----------------------')
score = knn_pipe.score(X_test_ove, y_test_ove)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_ove, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test_ove, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
-----------------------KNN Over----------------------
Accuracy Score:  0.8728340254904768
Weighted-F1: 0.8717628106865696
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.89      0.98      0.93       337
           1       1.00      0.98      0.99       325
           2       0.97      0.98      0.98       300
           3       0.87      0.64      0.74       315
           4       0.91      0.99      0.95       319
           5       0.87      0.88      0.88       309
           6       0.98      0.99      0.98       322
           7       0.91      0.79      0.85       327
           8       0.72      0.77      0.75       306
           9       0.63      0.68      0.65       321
          10       0.69      0.68      0.69       323
          11       0.84      0.92      0.88       280
          12       0.79      0.69      0

In [32]:
param_grid_knn = [{
    'knn__leaf_size': list(range(1, 15)),
    'knn__n_neighbors': list(range(1, 9)),
    'knn__p': [1, 2],
}]

In [33]:
knnModel_grid = GridSearchCV(estimator=knn_pipe, param_grid=param_grid_knn, verbose=1, cv=10, n_jobs=-1, scoring='f1_weighted')
knnModel_grid.fit(X_train_ove, y_train_ove)
print(knnModel_grid.best_estimator_)
print(f'Best Score: {knnModel_grid.best_score_}')

Fitting 10 folds for each of 224 candidates, totalling 2240 fits
Pipeline(steps=[('std_slc', StandardScaler()),
                ('knn', KNeighborsClassifier(leaf_size=1, n_neighbors=1, p=1))])
Best Score: 0.9221323183534441


In [34]:
knn_pipe = knnModel_grid.best_estimator_
clf = knn_pipe.fit(X_train_ove, y_train_ove)

pred_labels = knn_pipe.predict(X_test_ove)

print('Classes: ', clf.classes_)

print('----------------------KNN Over Best--------------------')
score = knn_pipe.score(X_test_ove, y_test_ove)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_ove, pred_labels, average='weighted')}")
print(f"Macro-F1: {f1_score(y_test_ove, pred_labels, average='macro')}")
print('--------------------------------------------------------')

print(classification_report(y_test_ove, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
----------------------KNN Over Best--------------------
Accuracy Score:  0.9251038235715309
Weighted-F1: 0.9245386376074354
Macro-F1: 0.9244371880041308
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.92      0.99      0.96       337
           1       1.00      1.00      1.00       325
           2       0.99      0.99      0.99       300
           3       0.90      0.74      0.82       315
           4       0.96      1.00      0.98       319
           5       0.90      0.90      0.90       309
           6       0.99      0.99      0.99       322
           7       0.94      0.84      0.89       327
           8       0.87      0.88      0.87       306
           9       0.76      0.75      0.76       321
          10       0.75      0.82      0.78       323
          11       0.92      0.96      0.94       280
         

##Support Vector Machines

In [35]:
svm_pipe = Pipeline(steps=[('std_slc', StandardScaler()),
                ('svm_reg', SVC())])

In [36]:
clf = svm_pipe.fit(X_train_ove, y_train_ove)

pred_labels = svm_pipe.predict(X_test_ove)

print('Classes: ', clf.classes_) 

print('------------------------SVM Over------------------------')
score = svm_pipe.score(X_test_ove, y_test_ove)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_ove, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test_ove, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
------------------------SVM Over------------------------
Accuracy Score:  0.6921094085636545
Weighted-F1: 0.676734149126241
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.95      0.47      0.63       337
           1       1.00      0.98      0.99       325
           2       0.75      0.95      0.84       300
           3       0.68      0.46      0.55       315
           4       0.62      0.72      0.67       319
           5       0.86      0.91      0.89       309
           6       0.77      0.99      0.87       322
           7       0.72      0.73      0.73       327
           8       0.42      0.54      0.47       306
           9       0.61      0.20      0.30       321
          10       0.51      0.72      0.60       323
          11       0.54      0.54      0.54       280
          12       0.50      0.39     

In [37]:
svm_param_grid = {'svm_reg__C': [980, 1000, 1020],
              'svm_reg__gamma': [1, 0.6, 0.3],
              'svm_reg__kernel': ['rbf']}

In [38]:
svmModel_grid = GridSearchCV(estimator=svm_pipe, param_grid=svm_param_grid, verbose=1, cv=3, n_jobs=-1, scoring='f1_weighted')
svmModel_grid.fit(X_train_ove, y_train_ove)
print(svmModel_grid.best_estimator_)
print(f'Best Score: {svmModel_grid.best_score_}')

Fitting 3 folds for each of 9 candidates, totalling 27 fits
Pipeline(steps=[('std_slc', StandardScaler()),
                ('svm_reg', SVC(C=1020, gamma=1))])
Best Score: 0.9008786246667243


In [39]:
svm_pipe = svmModel_grid.best_estimator_

clf = svm_pipe.fit(X_train_ove, y_train_ove)

pred_labels = svm_pipe.predict(X_test_ove)

print('Classes: ', clf.classes_) 

print('----------------------SVM over Best--------------------')
score = svm_pipe.score(X_test_ove, y_test_ove)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_ove, pred_labels, average='weighted')}")
print('--------------------------------------------------------')

print(classification_report(y_test_ove, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
----------------------SVM over Best--------------------
Accuracy Score:  0.9113561506515824
Weighted-F1: 0.9113116207796366
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.92      0.84      0.88       337
           1       1.00      0.99      0.99       325
           2       0.99      0.97      0.98       300
           3       0.82      0.76      0.79       315
           4       0.90      0.97      0.93       319
           5       0.91      0.91      0.91       309
           6       0.99      0.99      0.99       322
           7       0.94      0.89      0.92       327
           8       0.77      0.81      0.79       306
           9       0.72      0.82      0.77       321
          10       0.85      0.76      0.80       323
          11       0.88      0.95      0.91       280
          12       0.88      0.80     

##Random Forest

In [65]:
rf_pipe = Pipeline(steps=[('std_slc', StandardScaler()),
                ('ran_forest',
                 RandomForestClassifier())])

In [66]:
clf_d = rf_pipe.fit(X_train_ove, y_train_ove)

pred_labels = rf_pipe.predict(X_test_ove)

print('Classes: ', clf_d.classes_)

print('-------------------Random Forrest Over-----------------')
score = rf_pipe.score(X_test_ove, y_test_ove)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_ove, pred_labels, average='weighted')}")
print(f"Macro-F1: {f1_score(y_test_ove, pred_labels, average='macro')}")
print('--------------------------------------------------------')

print(classification_report(y_test_ove, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
-------------------Random Forrest Over-----------------
Accuracy Score:  0.9692109408563655
Weighted-F1: 0.969112490332675
Macro-F1: 0.9689233470647121
--------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       337
           1       1.00      0.99      0.99       325
           2       0.99      1.00      1.00       300
           3       0.92      0.89      0.90       315
           4       0.99      1.00      0.99       319
           5       0.94      0.95      0.95       309
           6       1.00      1.00      1.00       322
           7       0.95      0.95      0.95       327
           8       0.92      0.91      0.92       306
           9       0.89      0.92      0.91       321
          10       0.93      0.91      0.92       323
          11       0.98      0.99      0.99       280
          

In [56]:
rf_pipe = Pipeline(steps=[('std_slc', StandardScaler()),
                ('ran_forest',
                 RandomForestClassifier(max_depth=96, max_features='sqrt',
                                        min_samples_split=5,
                                        n_estimators=1450))])

In [57]:
clf = rf_pipe.fit(X_train_ove, y_train_ove)

pred_labels = rf_pipe.predict(X_test_ove)

print('Classes: ', clf.classes_)

print('-----------------Random Forrest Over Best---------------')
score = rf_pipe.score(X_test_ove, y_test_ove)
print('Accuracy Score: ', score)
print(f"Weighted-F1: {f1_score(y_test_ove, pred_labels, average='weighted')}")
print(f"Macro-F1: {f1_score(y_test_ove, pred_labels, average='macro')}")
print('--------------------------------------------------------')

print(classification_report(y_test_ove, pred_labels))

Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
-----------------Random Forrest Over Best---------------
Accuracy Score:  0.9666332521838751
Weighted-F1: 0.9665148900947244
Macro-F1: 0.9663303880296662
--------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       337
           1       1.00      0.99      0.99       325
           2       0.99      1.00      1.00       300
           3       0.92      0.90      0.91       315
           4       0.99      1.00      1.00       319
           5       0.92      0.95      0.93       309
           6       1.00      0.99      1.00       322
           7       0.94      0.94      0.94       327
           8       0.93      0.90      0.92       306
           9       0.89      0.92      0.90       321
          10       0.94      0.90      0.92       323
          11       0.99      0.99      0.99       280
        

In [86]:
u_pred_labels = rf_pipe.predict(test_data[["GPStime", "peakFreq", "snr", "duration", "ifo_num"]])
type(u_pred_labels)

numpy.ndarray

In [87]:
u_pred_labels = le.inverse_transform(u_pred_labels)

In [88]:
df_result = pd.DataFrame(u_pred_labels)
df_result.head()

,0
0,Koi_Fish
1,Blip
2,Helix
3,Blip
4,Blip


In [89]:
df_result.shape

(667, 1)

In [90]:
df_final = train_label[0][:667]
df_final = pd.DataFrame(df_final)

In [91]:
df_final[1] = df_result[0]
df_final.head()

,0,1
0,d1,Koi_Fish
1,d2,Blip
2,d3,Helix
3,d4,Blip
4,d5,Blip


In [92]:
df_final.tail()

,0,1
662,d663,Koi_Fish
663,d664,Helix
664,d665,Scattered_Light
665,d666,Violin_Mode
666,d667,Blip


In [93]:
df_final.to_csv('Kavin_R_V_Final_Project_normal.csv', index=False, )